In [1]:
from scipy import stats
from sklearn.metrics import accuracy_score, average_precision_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression, SGDClassifier, PassiveAggressiveClassifier, Lasso
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold
from sklearn import preprocessing
from sklearn import tree
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, IsolationForest, VotingClassifier
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, Normalizer, PolynomialFeatures
from sklearn.decomposition import PCA, KernelPCA, FastICA, SparsePCA
from sklearn.pipeline import Pipeline
from sklearn.cluster import FeatureAgglomeration
from sklearn.kernel_approximation import RBFSampler
# from sklearn.preprocessing importabs
from sklearn.impute import SimpleImputer
import random
import pandas as pd
import numpy as np
import warnings
import statsmodels.api as sm
warnings.filterwarnings('ignore')


In [2]:
dataset = pd.read_excel('Connect_Mobile__Attrition_Data_file.xlsx')


In [3]:
print(dataset.describe())
print(dataset.dtypes)
print(dataset.columns.values)

def get_data(target = 'label'):
    dataset = pd.read_excel('Connect_Mobile__Attrition_Data_file.xlsx')
    #dataset = dataset.iloc[:500,:]
    X = dataset.drop([target], axis=1, inplace=False)
    y = (dataset[[target]])*1
    return X,y

         active_cust         income    expenditure  months_on_network  \
count  100000.000000  100000.000000  100000.000000      100000.000000   
mean        0.578590    1855.279910    1409.669662          58.402450   
std         0.493787    2712.385257    2174.330870          18.643736   
min         0.000000    1130.000000     904.000000         -20.000000   
25%         0.000000    1674.000000    1248.650000          47.000000   
50%         1.000000    1805.000000    1358.400000          56.000000   
75%         1.000000    1907.000000    1494.400000          70.000000   
max         1.000000  229821.000000  183856.800000         449.000000   

       Num_complaints  number_plan_changes      relocated   monthly_bill  \
count   100000.000000        100000.000000  100000.000000  100000.000000   
mean         4.012840             1.385410       0.123480      95.816450   
std          0.285265             1.617101       0.328989      50.161327   
min          3.000000             0.00

In [4]:

X,y           = get_data(target = 'active_cust')
logit = sm.Logit(y, X)

# fit the model
result = logit.fit()
# summary report
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.501675
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:            active_cust   No. Observations:               100000
Model:                          Logit   Df Residuals:                    99991
Method:                           MLE   Df Model:                            8
Date:                Thu, 18 Feb 2021   Pseudo R-squ.:                  0.2630
Time:                        22:11:41   Log-Likelihood:                -50168.
converged:                       True   LL-Null:                       -68074.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
income                         0.0029   5.32e-05     54.041      0.000      

In [5]:

def Run_benchmark_classifiers(X, y, Model_list, Metrics_list, num_folds):
    text_file = open("../nik", "w")  # Open a file to store results

    ## Preprocessing step
    # X        = preprocessing.scale(X) # scaling
    X = preprocessing.normalize(X)  # normalization
    # pipeline = Pipeline([('scaling', Normalizer()), ('pca', PCA(n_components=4))]) # create a pipeline
    # pipeline = Pipeline([('scaling', Normalizer()), ('fast_ica', FastICA(n_components=4))])
    # pipeline = Pipeline([('scaling', Normalizer()), ('sparse_pca', SparsePCA(n_components=4))])
    # X        = pipeline.fit_transform(X) # In future versions split this between training and testing
    # poly     = PolynomialFeatures(interaction_only=True)
    # X        = poly.fit_transform(X)
    # agglo    = FeatureAgglomeration(n_clusters=4)
    # agglo.fit(X)
    # X        = agglo.transform(X)
    # RFF       = RBFSampler(gamma=1, random_state=1)
    # X         = RFF.fit_transform(X)

    for i in range(len(Model_list)):

        # Create an instantiation of the selected model
        if Model_list[i][0] == 'Logistic Regression':
            model = LogisticRegression()
        elif Model_list[i][0] == 'SGD':
            model = SGDClassifier(loss="hinge", penalty="l2")
        elif Model_list[i][0] == 'kNN':
            model = KNeighborsClassifier()
        elif Model_list[i][0] == 'Decision Tree':
            model = tree.DecisionTreeClassifier()
        elif Model_list[i][0] == 'Naive Bayes':
            if Model_list[i][1] == 'Gaussian':
                model = GaussianNB()
            if Model_list[i][1] == 'Bernoulli':
                model = BernoulliNB()
            if Model_list[i][1] == 'Multinomial':
                model = MultinomialNB()
        elif Model_list[i][0] == 'SVM':
            if Model_list[i][1] == 'Linear':
                model = svm.LinearSVC()
#             if Model_list[i][1] == 'Kernel':
#                 model = svm.SVC()
        elif Model_list[i][0] == 'Random Forest':
            model = RandomForestClassifier(n_estimators=20)
        elif Model_list[i][0] == 'Extra Trees':
            model = ExtraTreesClassifier(n_estimators=20)
        elif Model_list[i][0] == 'LDA':
            model = LinearDiscriminantAnalysis()
        elif Model_list[i][0] == 'QDA':
            model = QuadraticDiscriminantAnalysis()
        elif Model_list[i][0] == 'Passive Agressive':
            model = PassiveAggressiveClassifier()
        elif Model_list[i][0] == 'AdaBoost':
            model = AdaBoostClassifier()
        elif Model_list[i][0] == 'Bagging':
            model = BaggingClassifier()
        elif Model_list[i][0] == 'Gradient Boosting':
            model = GradientBoostingClassifier()
        elif Model_list[i][0] == 'Isolation Forest':
            model = IsolationForest()
        elif Model_list[i][0] == 'XGBoost':
            model = xgb.XGBClassifier()
        elif Model_list[i][0] == 'LASSO':
            model = Lasso()
        elif Model_list[i][0] == 'Dropout':
            layers = [Layer("Rectifier", units=100), Layer("Rectifier", units=100), Layer("Softmax")]
            model = Classifier(layers, dropout_rate=0.5)
        elif Model_list[i][0] == 'MLP':
            model = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=Model_list[i][1], random_state=1)
        elif Model_list[i][0] == 'AutoML':
            if Model_list[i][1] == 'Evolutionary':
                model = TPOTClassifier(generations=5, population_size=50, verbosity=2)
            if Model_list[i][1] == 'Bayesian':
                model = TPOTClassifier(generations=5, population_size=50, verbosity=2)

        # Compute the set of metrics via K-fold cross validation
        # -------------------------------------------------------
        if Model_list[i][0] == 'Naive Bayes':
            text_file.write("**Performance of %s %s:" % (Model_list[i][1], Model_list[i][0]))
            print("Performance of %s %s:" % (Model_list[i][1], Model_list[i][0]))
        else:
            text_file.write("**Performance of %s:" % Model_list[i][0])
            print("Performance of %s:" % Model_list[i][0])
        for k in range(len(Metrics_list)):
            scores_val = cross_val_score(model, X, y, cv=StratifiedKFold(n_splits=num_folds), scoring=Metrics_list[k])
            if k == len(Metrics_list) - 1:
                text_file.write(
                    "%s = %0.4f (+/- %0.4f)**\n" % (Metrics_list[k], scores_val.mean(), scores_val.std() * 2))
                print("%s = %0.4f (+/- %0.4f)\n" % (Metrics_list[k], scores_val.mean(), scores_val.std() * 2))
            else:
                text_file.write("%s = %0.4f (+/- %0.4f)," % (Metrics_list[k], scores_val.mean(), scores_val.std() * 2))
                print("%s = %0.4f (+/- %0.4f)" % (Metrics_list[k], scores_val.mean(), scores_val.std() * 2))
    text_file.close()
    return

In [ ]:
num_folds     = 10  
Model_list    = [['Logistic Regression'],['SGD'],['kNN'],
                 ['Decision Tree'],['Naive Bayes','Gaussian'],['Naive Bayes','Bernoulli'],
                 ['Naive Bayes','Multinomial'],['SVM','Linear'],['SVM','Kernel'],['QDA'],
                 ['Random Forest'], ['Extra Trees'],['LDA'],['Passive Agressive'],
                 ['AdaBoost'],['Bagging'],
                 ['Gradient Boosting'],['XGBoost'],['MLP',(100,100)]]     # List of models and parameters
Metrics_list  = ['roc_auc','recall','precision','accuracy','average_precision']
                                                                          # List of metrics for benchmark evaluation
    
Run_benchmark_classifiers(X,y,Model_list,Metrics_list,num_folds)

Performance of Logistic Regression:
roc_auc = 0.6854 (+/- 0.4361)
recall = 0.8062 (+/- 0.5154)
precision = 0.6223 (+/- 0.2753)
accuracy = 0.6083 (+/- 0.2812)
average_precision = 0.7459 (+/- 0.3085)

Performance of SGD:
roc_auc = 0.6851 (+/- 0.4277)
recall = 1.0000 (+/- 0.0000)
precision = 0.5786 (+/- 0.0001)
accuracy = 0.5786 (+/- 0.0001)
average_precision = 0.7459 (+/- 0.3033)

Performance of kNN:
roc_auc = 0.6853 (+/- 0.2962)
recall = 0.7047 (+/- 0.3907)
precision = 0.7122 (+/- 0.2360)
accuracy = 0.6554 (+/- 0.2506)
average_precision = 0.7165 (+/- 0.2264)

Performance of Decision Tree:
roc_auc = 0.6769 (+/- 0.2109)
recall = 0.6624 (+/- 0.3458)
precision = 0.7553 (+/- 0.2105)
accuracy = 0.6749 (+/- 0.2295)
average_precision = 0.7018 (+/- 0.1670)

Performance of Gaussian Naive Bayes:
roc_auc = 0.8063 (+/- 0.1929)
recall = 0.9560 (+/- 0.0836)
precision = 0.6581 (+/- 0.1507)
accuracy = 0.6776 (+/- 0.1864)
average_precision = 0.8120 (+/- 0.1791)

Performance of Bernoulli Naive Bayes:
roc_

In [21]:

def variable_importance(X,y):
    scores = []
    X = preprocessing.normalize(X)
    model = GradientBoostingClassifier()
#     model = LogisticRegression()
    
    for i in range(X.shape[1]):
        #data = np.delete(X, i, axis=1)
        data = X[:,i]
        data = data[:,np.newaxis]
        scores_val = cross_val_score(model, data, y, cv=StratifiedKFold(n_splits=3), scoring='roc_auc')
        scores.append(scores_val.mean())
        
        #print("AUC for %s removed = %0.4f (+/- %0.4f)" % (dataset.columns.values[4+i], scores_val.mean(), 
        #                                                    scores_val.std()))
        print("AUC with %s only = %0.4f (+/- %0.4f)" % (dataset.columns.values[4+i], scores_val.mean(), 
                                                            scores_val.std()))
    return scores

In [22]:
X,y           = get_data(target = 'active_cust')

scores = variable_importance(X,y)

AUC with Num_complaints only = 0.5795 (+/- 0.0492)
AUC with number_plan_changes only = 0.5793 (+/- 0.0434)
AUC with relocated only = 0.6610 (+/- 0.0671)
AUC with monthly_bill only = 0.5147 (+/- 0.0402)
AUC with technical_issues_per_month only = 0.6097 (+/- 0.1104)
AUC with Speed_test_result only = 0.5982 (+/- 0.0517)


IndexError: index 10 is out of bounds for axis 0 with size 10